# Fill your name and student number in here:
---
Name: Seyed Ali Mirferdos
<br>
Student Number: 99201465

In [204]:
r = 5
b = 20
f = 100
d = 300
N = 100_000

In [ ]:
import numpy as np
np.random.seed(65)

### Complete this file and use it in your future assignments.
---

## 1. Create Fake Data

In [ ]:
def create_fake_data(N, d):
    # todo: complete this section
    # returns X array with shape (N, d)
    return np.random.uniform(low=-100, high=100 ,size=(N, d))

In [ ]:
# Test
create_fake_data(4, 3)

array([[ -8.85181887, -78.57677411,   6.5145157 ],
       [-10.78503777, -64.05595655,  97.00810543],
       [ 32.64529855,  26.02567699,  10.05412916],
       [-94.0125269 , -78.84176529, -30.03826354]])

## 2. Cosine Distance and k-Nearest Neighours

In [ ]:
def cosine_distance(x, y):
    # todo: complete this section
    # returns cosine distance between x and y
    return 1 - np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))

In [ ]:
# Test
print(cosine_distance(np.array([1, 2, 4, 6]), np.array([1, 2, 4, 6])))
print(cosine_distance(np.array([1, 2, 4, 6]), np.array([1, 2, 5, 6])))
print(cosine_distance(np.array([1, 2, 4, 6]), np.array([1, 10, 15, 6])))

0.0
0.005464157642812495
0.18549426108801514


In [ ]:
def find_k_nearest_neighbours(X, q, k):
    # todo: complete this section
    # returns indexes of the k-nearest-neighbours of vector q
    dist = np.array([cosine_distance(i, q) for i in X])
    return dist.argsort()[:k]

In [ ]:
# Test
x1 = create_fake_data(4, 3)
print("x1:", x1)
q1 = np.array([50, -70, 80])
print("q1", q1)
print("result:", find_k_nearest_neighbours(x1, q1, 2))

x1: [[-98.25528056  30.77961656  54.53188162]
 [-95.88707323 -62.70208122  -5.74335631]
 [ 85.89175807  19.47597203  -0.67111476]
 [-98.53487268  12.80261809 -45.2913393 ]]
q1 [ 50 -70  80]
result: [2 1]


In [ ]:
x2 = create_fake_data(N, d)
q2 = np.random.uniform(low=-100, high=100 ,size=d)
k2 = 20

In [ ]:
%time find_k_nearest_neighbours(x2, q2, k2)

CPU times: user 1.47 s, sys: 8.55 ms, total: 1.48 s
Wall time: 1.49 s


array([57985, 95498, 58604,  4283, 24387, 98238, 48727, 45637, 82210,
       26363, 12080, 87748,  5840,  4215,   221, 66352, 49162, 64427,
       83312, 65400])

## 3. Signer class

In [51]:
class Signer:
    def __init__(self, f, d):
        # todo: complete this section
        # initiates f random vectors with dimension = d
        self.hyper_planes = create_fake_data(f, d)
        
    def hash_point(self, x):
        # x: (1, d)
        # todo: complete this section
        # returns list of numbers in {0, 1} in shape (1, f)
        y = []
        for p in self.hyper_planes:
          dot_product = np.dot(x, p)
          binary_res = int(dot_product > 0)
          y.append(binary_res)
        return np.array(y)

In [53]:
# Test
signer = Signer(f, d)
signer.hash_point(q2)

array([1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0])

## 4. LSHIndex

In [208]:
class LSHIndex:
    def __init__(self, f, b, d, data):
        # todo: complete this section
        # initiates empty hash tables
        self.f = f
        self.b = b
        self.f_b = f // b
        self.signer = Signer(f, d)
        self.hash_tables = [{} for i in range(b)]
        self.data = data
    
    def index(self, x_index):
        # The signature of the method is changed following the discussions in 
        # the group.
        # todo: complete this section
        # returns a boolean, indicating success or failure
        x = self.data[x_index, :]
        x_hash = self.signer.hash_point(x) # the output would be (1, f)
        parts = np.array_split(x_hash, b)
        
        for i in range(b):
          curr_table = self.hash_tables[i]
          part = int(''.join(parts[i].astype(str)), 2)
          
          if part in curr_table:
            curr_table[part].append(x_index)
          else:
            curr_table[part] = [x_index]
    
    def index_all(self):
      for i in range(self.data.shape[0]):
        self.index(i)

    def query(self, q, k):
        # todo: complete this section
        # q: (1, d)
        # returns k-approximate-nearest-neighbours of vector q
        q_hash = self.signer.hash_point(q)
        parts = np.array_split(q_hash, b)
        candidates = set()

        for i in range(b):
          curr_table = self.hash_tables[i]
          part = int(''.join(parts[i].astype(str)), 2)
          if part in curr_table:
              candidates.update(curr_table[part])
        
        x = self.data[list(candidates)]
        result = find_k_nearest_neighbours(x, q, k)
        return result

In [209]:
lsh = LSHIndex(f, b, d, x2)
lsh.index_all()

In [211]:
len(lsh.hash_tables[1][14])

3362

In [212]:
%time lsh.query(q2, k2)

CPU times: user 765 ms, sys: 0 ns, total: 765 ms
Wall time: 766 ms


array([27347, 45015, 27625, 11489, 46346, 22968, 21501, 12396,  5666,
       41394,  2700,    87, 31295, 23188, 30349, 39308, 30840, 13114,
       27287, 38966])

The wall time for previous part was 1.49 s whereby it's 766 ms here which is a 50% reduction and is caused because of using LSH indexing as it indexes the candidates and find the similarity in the candidate space which is a far more reduced space than the main space.